#STEP 01. 기존 데이터 개선

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(",", '', x)
    
    if x =='':
        return 0
    else:
        return int(x)


PATH = '/content/gdrive/MyDrive/dacon/'

In [3]:
df = pd.read_csv(PATH + "preprocessing_02.csv")
df.describe()

,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,result,result_sum,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,450000.000000,4.500000e+05,450000.000000,450000.000000
mean,17499.500000,15.500000,0.055073,0.061813,0.072660,3.124327,2.384547,0.125220,0.128809,0.000393,0.000253,0.333478,0.782493,0.819091,0.059180,0.621493,4.042589,3.145149,0.116207,0.006673,0.001680,0.005213,0.001544,4.589616,5.607133,0.098227,0.002953,1.346742,1.386953,1.395500,0.000991,4.968962,2.863873,0.448211,0.409591,0.025013,0.023453,0.023504,0.011498,0.040558,1.828616,0.259016,0.086376,0.012064,0.183800,2.054596,0.014524,3.043840,37.794551,0.920960,2.943249,0.021204,6.109153,8.618243e+01,0.003922,0.004738
std,4330.131821,8.655451,0.350321,4.323228,4.116721,101.735662,17.119533,2.946216,2.701991,0.043177,0.033927,53.879771,7.570427,7.827941,6.357724,17.166959,5.225690,4.829078,0.946797,0.116438,0.058094,0.091915,0.055496,32.240711,48.802248,3.219677,0.145458,1.418872,84.740592,84.754038,0.034110,16.571571,21.447965,0.788737,17.483695,0.789795,0.161138,0.161482,0.190231,1.169040,5.482665,2.080207,0.305519,0.111809,0.422837,367.591850,0.404088,538.950934,1581.025620,18.570483,100.336538,2.013909,1122.075962,5.599574e+03,0.083214,0.105745
min,10000.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,13749.750000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,17499.500000,15.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,21249.250000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,6.000000,4.000000,0.000000,

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 56 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   user_id     450000 non-null  int64
 1   days        450000 non-null  int64
 2   err_1       450000 non-null  int64
 3   err_2       450000 non-null  int64
 4   err_3       450000 non-null  int64
 5   err_4       450000 non-null  int64
 6   err_5       450000 non-null  int64
 7   err_6       450000 non-null  int64
 8   err_7       450000 non-null  int64
 9   err_8       450000 non-null  int64
 10  err_9       450000 non-null  int64
 11  err_10      450000 non-null  int64
 12  err_11      450000 non-null  int64
 13  err_12      450000 non-null  int64
 14  err_13      450000 non-null  int64
 15  err_14      450000 non-null  int64
 16  err_15      450000 non-null  int64
 17  err_16      450000 non-null  int64
 18  err_17      450000 non-null  int64
 19  err_18      450000 non-null  int64
 20  err_

In [14]:
df["model_change"] = 0
df["fwver_change"] = 0
df["qual_fwver_change"] = 0
df

,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,result,result_sum,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change
0,10000,1,0,0,0,8,0,0,0,0,0,0,2,2,0,0,4,4,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000,2,0,0,0,12,0,0,0,0,0,0,1,1,0,0,6,6,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000,3,0,0,4,8,0,0,0,0,0,4,6,6,0,4,6,4,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000,4,0,0,2,6,0,0,0,0,0,2,3,3,0,2,4,3,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000,5,0,0,0,10,0,0,0,0,0,0,2,2,0,0,3,5,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,24999,26,0,0,0,10,0,0,0,0,0,0,0,0,0,0,6,6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
449996,24999,27,0,0,0,8,0,0,0,0,0,0,1,1,0,0,4,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
449997,24999,28,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
449998,24999,29,0,0,0,4,1,0,0,0,0,0,1,1,0,0,5,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
err = pd.read_csv(PATH + "train_err_data.csv")
err['datetime'] = err['time'].apply(make_datetime)

min_day = err.datetime.min().date()

err['days'] = err.loc[:,'datetime'].dt.date - min_day
err['days'] = err['days'].dt.days
display(err)

,user_id,time,model_nm,fwver,errtype,errcode,datetime,days
0,10000,20201101025616,model_3,05.15.2138,15,1,2020-11-01 02:00:00,1
1,10000,20201101030309,model_3,05.15.2138,12,1,2020-11-01 03:00:00,1
2,10000,20201101030309,model_3,05.15.2138,11,1,2020-11-01 03:00:00,1
3,10000,20201101050514,model_3,05.15.2138,16,1,2020-11-01 05:00:00,1
4,10000,20201101050515,model_3,05.15.2138,4,0,2020-11-01 05:00:00,1
...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1,2020-11-30 16:00:00,30
16554659,24999,20201130172625,model_3,05.15.2138,16,1,2020-11-30 17:00:00,30
16554660,24999,20201130172625,model_3,05.15.2138,4,0,2020-11-30 17:00:00,30
16554661,24999,20201130172631,model_3,05.15.2138,4,0,2020-11-30 17:00:00,30


In [7]:
qual = pd.read_csv(PATH + "pre_qual.csv")

qual['datetime'] = qual['time'].apply(make_datetime)

min_day = qual.datetime.min().date()

qual['days'] = qual.loc[:,'datetime'].dt.date - min_day
qual['days'] = qual['days'].dt.days
display(qual)

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,datetime,days
0,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,29
1,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,29
2,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,29
3,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,29
4,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0,0,0,0,0,1,0,0,0,0,17,0,0,2020-11-24 03:00:00,24
828620,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,17,0,0,2020-11-24 03:00:00,24
828621,20201124033000,24997,04.22.1778,0,0,0,0,0,3,0,0,0,0,17,0,0,2020-11-24 03:00:00,24
828622,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,17,0,0,2020-11-24 03:00:00,24


In [8]:
err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 8 columns):
 #   Column    Dtype         
---  ------    -----         
 0   user_id   int64         
 1   time      int64         
 2   model_nm  object        
 3   fwver     object        
 4   errtype   int64         
 5   errcode   object        
 6   datetime  datetime64[ns]
 7   days      int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 1010.4+ MB


In [9]:
# model_change count

mc_list = []

for i in tqdm(range(10000,25000)) :
  mc_list.append(len(set(err[err["user_id"] == i]["model_nm"])))

100%|██████████| 15000/15000 [04:58<00:00, 50.29it/s]


In [10]:
# fwver_change count - 1

fwver_list = []

for i in tqdm(range(10000,25000)) :
  fwver_list.append(len(set(err[err["user_id"] == i]["fwver"])))

100%|██████████| 15000/15000 [04:56<00:00, 50.51it/s]


In [11]:
# fwver_change count - 2

fwver_list_2 = []

for i in tqdm(range(10000,25000)) :
  fwver_list_2.append(len(set(qual[qual["user_id"] == i]["fwver"])))

100%|██████████| 15000/15000 [00:23<00:00, 633.81it/s]


In [17]:
for i in range(15000) :
    df["model_change"][i*30+29] = mc_list[i] - 1
    df["fwver_change"][i*30+29] = fwver_list[i] - 1
    df["qual_fwver_change"][i*30+29] = fwver_list_2[i] - 1

df

,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,result,result_sum,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change
0,10000,1,0,0,0,8,0,0,0,0,0,0,2,2,0,0,4,4,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000,2,0,0,0,12,0,0,0,0,0,0,1,1,0,0,6,6,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000,3,0,0,4,8,0,0,0,0,0,4,6,6,0,4,6,4,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000,4,0,0,2,6,0,0,0,0,0,2,3,3,0,2,4,3,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000,5,0,0,0,10,0,0,0,0,0,0,2,2,0,0,3,5,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,24999,26,0,0,0,10,0,0,0,0,0,0,0,0,0,0,6,6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
449996,24999,27,0,0,0,8,0,0,0,0,0,0,1,1,0,0,4,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
449997,24999,28,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
449998,24999,29,0,0,0,4,1,0,0,0,0,0,1,1,0,0,5,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
df[:30]

,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,result,result_sum,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change
0,10000,1,0,0,0,8,0,0,0,0,0,0,2,2,0,0,4,4,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000,2,0,0,0,12,0,0,0,0,0,0,1,1,0,0,6,6,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000,3,0,0,4,8,0,0,0,0,0,4,6,6,0,4,6,4,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000,4,0,0,2,6,0,0,0,0,0,2,3,3,0,2,4,3,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000,5,0,0,0,10,0,0,0,0,0,0,2,2,0,0,3,5,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,10000,6,0,0,0,12,0,0,0,0,0,0,1,1,0,0,5,6,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,10000,7,0,0,4,12,0,0,0,0,0,4,6,6,0,4,4,6,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,10000,8,0,0,2,8,0,0,0,0,0,2,3,3,0,2,5,6,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,10000,9,0,0,0,6,0,0,0,0,0,0,0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,10000,10,0,0,0,10,0,0,0,0,0,0,0,0,0,0,7,5,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
df.to_csv(PATH + "preprocessing_03.csv", index=False, encoding="utf-8-sig")

# AUTOML 이용

##1. TARGET을 바로 예측하기(RESULT)

In [31]:
df = pd.read_csv(PATH + "preprocessing_03.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 59 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   user_id            450000 non-null  int64
 1   days               450000 non-null  int64
 2   err_1              450000 non-null  int64
 3   err_2              450000 non-null  int64
 4   err_3              450000 non-null  int64
 5   err_4              450000 non-null  int64
 6   err_5              450000 non-null  int64
 7   err_6              450000 non-null  int64
 8   err_7              450000 non-null  int64
 9   err_8              450000 non-null  int64
 10  err_9              450000 non-null  int64
 11  err_10             450000 non-null  int64
 12  err_11             450000 non-null  int64
 13  err_12             450000 non-null  int64
 14  err_13             450000 non-null  int64
 15  err_14             450000 non-null  int64
 16  err_15             450000 non-null  in

In [5]:
!pip install pycaret

In [32]:
df_new = pd.DataFrame()
df.columns

for i in tqdm(df.columns) :
  tmp_list = []
  for j in tqdm(range(15000)) :
    tmp_list.append(df[i][j*30+29])
  df_new[i] = pd.Series(tmp_list)
df_new

100%|██████████| 15000/15000 [00:00<00:00, 178286.30it/s]

100%|██████████| 15000/15000 [00:00<00:00, 169293.09it/s]

100%|██████████| 15000/15000 [00:00<00:00, 138362.55it/s]

100%|██████████| 15000/15000 [00:00<00:00, 171885.49it/s]

100%|██████████| 15000/15000 [00:00<00:00, 177421.53it/s]

100%|██████████| 15000/15000 [00:00<00:00, 175209.45it/s]

100%|██████████| 15000/15000 [00:00<00:00, 166669.47it/s]

100%|██████████| 15000/15000 [00:00<00:00, 167569.74it/s]

100%|██████████| 15000/15000 [00:00<00:00, 166549.02it/s]

100%|██████████| 15000/15000 [00:00<00:00, 172851.70it/s]

100%|██████████| 15000/15000 [00:00<00:00, 172698.91it/s]

100%|██████████| 59/59 [00:06<00:00,  9.62it/s]


,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,result,result_sum,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change
0,10000,30,0,0,0,12,0,0,0,0,0,0,1,1,0,0,6,6,0,0,0,0,0,0,0,0,0,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,8,0,0,0,0,0
1,10001,30,0,0,0,0,5,0,0,0,0,0,1,1,0,6,18,15,0,2,0,1,0,6,7,1,1,8,0,0,0,30,0,1,2,0,0,0,0,0,4,9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1
2,10002,30,0,0,0,12,0,0,0,0,0,0,1,1,0,0,4,5,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,8,0,0,0,0,0
3,10003,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,5,3,0,0,0,0,0,1,0,0,0,6,0,0,0,6,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1
4,10004,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,12,9,0,0,0,0,0,12,7,0,0,6,0,0,0,17,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,5,2,0,0,0,0,0,2,1,0,0,4,0,0,0,3,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14996,24996,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
14997,24997,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,2,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1
14998,24998,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1


In [33]:
df_2 = df_new[df_new.columns.difference(["user_id", "result_sum"])]

In [37]:
df_2.shape

(15000, 57)

In [34]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

clf = setup(data = df_2, target = "result") 

,Description,Value
0,session_id,4347
1,Target,result
2,Target Type,Multiclass
3,Label Encoded,"0: 0, 1: 1, 2: 2"
4,Original Data,"(15000, 57)"
5,Missing Values,False
6,Numeric Features,24
7,Categorical Features,32
8,Ordinal Features,False
9,High Cardinality Features,False


In [38]:
best_5 = compare_models(sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9795,0.4318,0.2190,0.6251,0.5998,0.0539,0.0750,13.773
rf,Random Forest Classifier,0.9810,0.4292,0.2227,0.6501,0.6017,0.0756,0.1190,1.429
et,Extra Trees Classifier,0.9796,0.4281,0.2399,0.6987,0.6249,0.1003,0.1348,1.934
xgboost,Extreme Gradient Boosting,0.9800,0.3875,0.2278,0.6769,0.6164,0.0642,0.0961,26.074
knn,K Neighbors Classifier,0.9811,0.3274,0.2000,0.5775,0.5830,-0.0005,-0.0013,3.896
ada,Ada Boost Classifier,0.8292,0.2699,0.2420,0.5857,0.5147,0.0086,0.0161,0.992
ridge,Ridge Classifier,0.9810,0.0000,0.2367,0.7034,0.6201,0.1074,0.1496,0.282


In [39]:
blended = blend_models(estimator_list = best_5, fold = 5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9800,0.7774,0.3415,0.9672,0.9724,0.0415,0.0657
1,0.9810,0.7614,0.3332,0.9632,0.9720,-0.0009,-0.0030
2,0.9810,0.7635,0.3418,0.9697,0.9729,0.0451,0.0882
3,0.9819,0.7004,0.3423,0.9813,0.9734,0.0491,0.1587
4,0.9814,0.7575,0.3419,0.9727,0.9731,0.0471,0.1101
Mean,0.9810,0.7520,0.3401,0.9708,0.9727,0.0364,0.0840
SD,0.0006,0.0267,0.0035,0.0061,0.0005,0.0188,0.0532


In [ ]:
pred_holdout = predict_model(blended)
pred_holdout

In [ ]:
final_model = finalize_model(blended)
final_model

In [40]:
predictions = predict_model(blended, data = df_2)
predictions

,days,err_1,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_2,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_3,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_4,err_40,err_41,err_42,err_5,err_6,err_7,err_8,err_9,fwver_change,model_change,qual_fwver_change,quality_0,quality_1,quality_10,quality_11,quality_12,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,result,Label,Score
0,30,0,0,1,1,0,0,6,6,0,0,0,0,0,0,0,0,0,0,3,1,1,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,8,0,0,0,0,0,0,0.9915
1,30,0,0,1,1,0,6,18,15,0,2,0,0,1,0,6,7,1,1,8,0,0,0,0,30,0,1,2,0,0,0,0,0,0,4,9,0,5,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9741
2,30,0,0,1,1,0,0,4,5,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,5,0,0,0,0,0,0,0.9970
3,30,0,0,1,1,0,0,5,3,0,0,0,0,0,0,1,0,0,0,6,0,0,0,0,6,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9920
4,30,0,0,1,1,0,0,12,9,0,0,0,0,0,0,12,7,0,0,6,0,0,0,0,17,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,30,0,0,1,1,0,0,5,2,0,0,0,0,0,0,2,1,0,0,4,0,0,0,0,3,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9990
14996,30,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9926
14997,30,0,0,0,0,0,0,2,2,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9956
14998,30,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9958


In [ ]:
set(predictions["result"])

In [41]:
set(predictions["Label"])

{0, 1, 2}

In [42]:
set(predictions["Score"])

{0.9741,
 0.992,
 0.9756,
 0.976,
 0.9775,
 0.9882,
 0.8647,
 0.9379,
 0.9512,
 0.9924,
 0.9863,
 0.9894,
 0.9269,
 0.5793,
 0.8918,
 0.625,
 0.8156,
 0.9574,
 0.9468,
 0.773,
 0.6343,
 0.5093,
 0.5718,
 0.9375,
 0.9499,
 0.6542,
 0.8585,
 0.921,
 0.9835,
 0.9698,
 0.7335,
 0.7198,
 0.5948,
 0.5323,
 0.9866,
 0.8616,
 0.9272,
 0.9897,
 0.9791,
 0.9303,
 0.8541,
 0.6178,
 0.9166,
 0.5416,
 0.5447,
 0.9334,
 0.9197,
 0.9822,
 0.9471,
 0.9502,
 0.9853,
 0.874,
 0.968,
 0.9396,
 0.9259,
 0.6271,
 0.9564,
 0.9427,
 0.9458,
 0.897,
 0.772,
 0.9626,
 0.7126,
 0.702,
 0.9657,
 0.9794,
 0.9825,
 0.9063,
 0.9688,
 0.9551,
 0.9231,
 0.8469,
 0.9856,
 0.9094,
 0.9125,
 0.9887,
 0.9262,
 0.975,
 0.5512,
 0.9918,
 0.9781,
 0.8531,
 0.7906,
 0.9293,
 0.9949,
 0.9812,
 0.9461,
 0.9324,
 0.998,
 0.9843,
 0.9492,
 0.7968,
 0.9355,
 0.9523,
 0.6886,
 0.5148,
 0.8898,
 0.9554,
 0.8929,
 0.9417,
 0.8823,
 0.9585,
 0.9448,
 0.8198,
 0.6979,
 0.9784,
 0.576,
 0.9159,
 0.951,
 0.9815,
 0.9678,
 0.9541,
 0.669

In [43]:
set(predictions["Label"] * predictions["Score"])

{0.0,
 0.5067,
 0.5091,
 0.5093,
 0.5128,
 0.5221,
 0.5278,
 0.5323,
 0.5382,
 0.5398,
 0.5423,
 0.5447,
 0.5507,
 0.5511,
 0.5512,
 0.5568,
 0.5635,
 0.5665,
 0.5718,
 0.5732,
 0.576,
 0.5793,
 0.5825,
 0.5879,
 0.5906,
 0.591,
 0.5921,
 0.5948,
 0.5971,
 0.5995,
 0.6002,
 0.6022,
 0.6024,
 0.6025,
 0.6026,
 0.6032,
 0.6046,
 0.6112,
 0.6225,
 0.6228,
 0.6298,
 0.6307,
 0.633,
 0.6362,
 0.6401,
 0.642,
 0.6422,
 0.6441,
 0.6451,
 0.6458,
 0.6486,
 0.6502,
 0.6524,
 0.6538,
 0.6556,
 0.6578,
 0.6592,
 0.6645,
 0.6658,
 0.669,
 0.6695,
 0.6733,
 0.6767,
 0.6776,
 0.6795,
 0.6822,
 0.6827,
 0.6839,
 0.6843,
 0.6858,
 0.6886,
 0.6913,
 0.6925,
 0.6955,
 0.699,
 0.7006,
 0.7008,
 0.702,
 0.7041,
 0.7042,
 0.7044,
 0.705,
 0.7071,
 0.7102,
 0.7117,
 0.7121,
 0.7126,
 0.714,
 0.7149,
 0.7179,
 0.719,
 0.7198,
 0.7207,
 0.7228,
 0.7234,
 0.728,
 0.7293,
 0.7301,
 0.7309,
 0.7328,
 0.7331,
 0.7335,
 0.7345,
 0.7376,
 0.7381,
 0.7388,
 0.7399,
 0.7408,
 0.7426,
 0.7446,
 0.7458,
 0.7471,
 0.748

In [51]:
test_df = pd.read_csv(PATH + "test_preprocessing_02.csv")
test = pd.DataFrame()

for i in test_df.columns :
  tmp_list = []
  for j in range(14999) :
    tmp_list.append(test_df[i][j*30+29])
  test[i] = pd.Series(tmp_list)

test

,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change
0,30000,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,1,2,0
1,30001,30,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0
2,30002,30,0,0,0,0,1,0,0,0,0,0,1,1,0,0,7,4,0,0,0,0,0,4,3,0,0,3,0,0,0,7,0,1,0,0,0,0,0,0,8,0,1,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0
3,30003,30,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,1
4,30004,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,2,11,0,0.0,0,0.0,0,0,0,0,0,0,0,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,2,2,0,0,1,0,0,0,6,0,0,0,0,0,0,0,0,6,2,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0
14995,44995,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0
14996,44996,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,7,8,0,0,2,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,1
14997,44997,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,237,253,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,2,1,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0


In [52]:
predictions = predict_model(blended, data = test)
predictions

,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change,Label,Score
0,30000,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,1,2,0,0,0.9972
1,30001,30,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.9983
2,30002,30,0,0,0,0,1,0,0,0,0,0,1,1,0,0,7,4,0,0,0,0,0,4,3,0,0,3,0,0,0,7,0,1,0,0,0,0,0,0,8,0,1,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0.9594
3,30003,30,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,1,0,0.9955
4,30004,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,2,11,0,0.0,0,0.0,0,0,0,0,0,0,0,0,1,2,1,0,0.9992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,2,2,0,0,1,0,0,0,6,0,0,0,0,0,0,0,0,6,2,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.9973
14995,44995,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0.9917
14996,44996,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,7,8,0,0,2,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,1,0,0.9926
14997,44997,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,237,253,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,2,1,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0.8285


In [53]:
result_01 = predictions
result_01

,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change,Label,Score
0,30000,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,1,2,0,0,0.9972
1,30001,30,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.9983
2,30002,30,0,0,0,0,1,0,0,0,0,0,1,1,0,0,7,4,0,0,0,0,0,4,3,0,0,3,0,0,0,7,0,1,0,0,0,0,0,0,8,0,1,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0.9594
3,30003,30,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,1,0,0.9955
4,30004,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,2,11,0,0.0,0,0.0,0,0,0,0,0,0,0,0,1,2,1,0,0.9992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,2,2,0,0,1,0,0,0,6,0,0,0,0,0,0,0,0,6,2,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.9973
14995,44995,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0.9917
14996,44996,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,7,8,0,0,2,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,1,0,0.9926
14997,44997,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,237,253,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,2,1,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0.8285


In [54]:
set(result_01.Label)

{0, 1}

In [50]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0


In [72]:
for i in range(len(result_01) ) :
  if result_01["Label"][i] >= 1 :
    sample_submssion["problem"][i] = result_01["Score"][i]
  else :
    sample_submssion["problem"][i] = 1- result_01["Score"][i]

sample_submssion.problem = result_01.Label
sample_submssion

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0


In [65]:
sample_submssion[sample_submssion.problem == 1]

,user_id,problem
450,30450,1
952,30952,1
2297,32297,1
2854,32854,1
4187,34187,1
6304,36304,1
7409,37409,1
7562,37562,1
7650,37650,1
9269,39269,1


In [73]:
sample_submssion.to_csv(PATH + "answer.csv", index=False)

##2. result_sum을 예측한 후에, 그 결과값을 다시 분류


In [67]:
df = pd.read_csv(PATH + "preprocessing_03.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 59 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   user_id            450000 non-null  int64
 1   days               450000 non-null  int64
 2   err_1              450000 non-null  int64
 3   err_2              450000 non-null  int64
 4   err_3              450000 non-null  int64
 5   err_4              450000 non-null  int64
 6   err_5              450000 non-null  int64
 7   err_6              450000 non-null  int64
 8   err_7              450000 non-null  int64
 9   err_8              450000 non-null  int64
 10  err_9              450000 non-null  int64
 11  err_10             450000 non-null  int64
 12  err_11             450000 non-null  int64
 13  err_12             450000 non-null  int64
 14  err_13             450000 non-null  int64
 15  err_14             450000 non-null  int64
 16  err_15             450000 non-null  in

In [68]:
df_new = pd.DataFrame()
df.columns

for i in tqdm(df.columns) :
  tmp_list = []
  for j in tqdm(range(15000)) :
    tmp_list.append(df[i][j*30+29])
  df_new[i] = pd.Series(tmp_list)
df_new

100%|██████████| 15000/15000 [00:00<00:00, 171383.09it/s]

100%|██████████| 15000/15000 [00:00<00:00, 168734.62it/s]

100%|██████████| 59/59 [00:06<00:00,  9.37it/s]


,user_id,days,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,result,result_sum,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_change,fwver_change,qual_fwver_change
0,10000,30,0,0,0,12,0,0,0,0,0,0,1,1,0,0,6,6,0,0,0,0,0,0,0,0,0,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,8,0,0,0,0,0
1,10001,30,0,0,0,0,5,0,0,0,0,0,1,1,0,6,18,15,0,2,0,1,0,6,7,1,1,8,0,0,0,30,0,1,2,0,0,0,0,0,4,9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1
2,10002,30,0,0,0,12,0,0,0,0,0,0,1,1,0,0,4,5,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,8,0,0,0,0,0
3,10003,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,5,3,0,0,0,0,0,1,0,0,0,6,0,0,0,6,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1
4,10004,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,12,9,0,0,0,0,0,12,7,0,0,6,0,0,0,17,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,5,2,0,0,0,0,0,2,1,0,0,4,0,0,0,3,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14996,24996,30,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
14997,24997,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,2,0,0,0,2,0,0,0,4,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1
14998,24998,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1


In [69]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

df_2 = df_new[df_new.columns.difference(["user_id", "result"])]

clf = setup(data = df_2, target = "result_sum") 

,Description,Value
0,session_id,7636
1,Target,result_sum
2,Target Type,Multiclass
3,Label Encoded,"0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5"
4,Original Data,"(15000, 57)"
5,Missing Values,False
6,Numeric Features,22
7,Categorical Features,34
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:01:43
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.7083,0.0000,0.2607,0.6943,0.6363,0.1839,0.2601,0.074
lr,Logistic Regression,0.6903,0.0644,0.2411,0.6578,0.6053,0.1192,0.1843,10.268
ada,Ada Boost Classifier,0.6764,0.0530,0.2409,0.7156,0.6049,0.1452,0.2272,0.920
rf,Random Forest Classifier,0.6724,0.0631,0.2604,0.6268,0.6352,0.1715,0.1855,2.046
knn,K Neighbors Classifier,0.6460,0.0572,0.2453,0.5901,0.6043,0.0971,0.1059,3.770
dt,Decision Tree Classifier,0.6156,0.0520,0.2384,0.6028,0.6081,0.1324,0.1329,0.178
svm,SVM - Linear Kernel,0.5844,0.0000,0.1981,0.6185,0.5812,0.1210,0.1332,0.471
nb,Naive Bayes,0.1313,0.0559,0.2026,0.6053,0.2084,0.0096,0.0180,0.070
qda,Quadratic Discriminant Analysis,0.0005,0.0501,0.0225,0.0001,0.0002,0.0001,0.0006,0.285


In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5)

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(blended, data = df_2)

In [ ]:
test_df = pd.read_csv(PATH + "test_preprocessing_02.csv")
test = pd.DataFrame()

for i in test_df.columns :
  tmp_list = []
  for j in range(14999) :
    tmp_list.append(test_df[i][j*30+29])
  test[i] = pd.Series(tmp_list)

test

In [ ]:
predictions = predict_model(blended, data = test)

In [ ]:
result_01 = predictions

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
for i in range(len(result_01) ) :
  if result_01["Label"][i] >= 1 :
    sample_submssion["problem"][i] = result_01["Score"][i]
  else :
    sample_submssion["problem"][i] = 1- result_01["Score"][i]

sample_submssion.problem = result_01.Label
sample_submssion

In [ ]:
sample_submssion.to_csv(PATH + "answer.csv", index=False)